In [14]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm

In [2]:
tf.version.VERSION

'2.4.1'

In [3]:
tf.config.list_physical_devices('GPU')

2022-08-15 23:25:19.156248: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-15 23:25:19.157508: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-08-15 23:25:19.221507: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:19:00.0 name: NVIDIA GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.77GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-08-15 23:25:19.222025: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:1a:00.0 name: NVIDIA GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.77GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-08-15 23:25:19.222498: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:67

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

Bandwidth: 417.29GiB/s
2022-08-15 23:25:19.222960: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 3 with properties: 
pciBusID: 0000:68:00.0 name: NVIDIA GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.77GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-08-15 23:25:19.222992: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-08-15 23:25:19.225719: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-08-15 23:25:19.225810: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2022-08-15 23:25:19.227649: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2022-08-15 23:25:19.228029: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully

In [4]:
directory = '/home/charlene/15aug_val_dataset/'
df = pd.read_csv(directory+'val.csv',delimiter=' ')
df

,file_name,label
0,ILSVRC2012_val_00000001.JPEG,65
1,ILSVRC2012_val_00000002.JPEG,970
2,ILSVRC2012_val_00000003.JPEG,230
3,ILSVRC2012_val_00000004.JPEG,809
4,ILSVRC2012_val_00000005.JPEG,516
...,...,...
49995,ILSVRC2012_val_00049996.JPEG,283
49996,ILSVRC2012_val_00049997.JPEG,26
49997,ILSVRC2012_val_00049998.JPEG,232
49998,ILSVRC2012_val_00049999.JPEG,982


In [5]:
file_paths = df['file_name'].values  # 50000 files
file_paths

array(['ILSVRC2012_val_00000001.JPEG', 'ILSVRC2012_val_00000002.JPEG',
       'ILSVRC2012_val_00000003.JPEG', ...,
       'ILSVRC2012_val_00049998.JPEG', 'ILSVRC2012_val_00049999.JPEG',
       'ILSVRC2012_val_00050000.JPEG'], dtype=object)

In [6]:
labels= df['label'].values
labels

array([ 65, 970, 230, ..., 232, 982, 355])

In [7]:
tf.device('/gpu:0')

2022-08-15 23:25:24.029602: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-15 23:25:24.030862: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-15 23:25:24.434095: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:19:00.0 name: NVIDIA GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.77GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-08-15 23:25:24.434663: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:1a:00.0 name: NVIDIA GeForce RTX 2070 SUPER computeCapability: 7.5
co

ed dynamic library libcurand.so.10
2022-08-15 23:25:24.435946: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcusolver.so.10
2022-08-15 23:25:24.435968: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcusparse.so.10
2022-08-15 23:25:24.435990: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-08-15 23:25:24.439351: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1862] Adding visible gpu devices: 0, 1, 2, 3
2022-08-15 23:25:24.439419: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-08-15 23:25:26.260381: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1261] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-08-15 23:25:26.260407: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1267]      0 1 2 3 
2022-08-15 23

In [8]:
ds_train=tf.data.Dataset.from_tensor_slices((file_paths,labels))

In [9]:
def read_image(image_file, label):
    image= tf.io.read_file(directory+image_file)
    image=tf.image.decode_image(image, channels=3,dtype=tf.float32,expand_animations=False)
#     print('Image type: ',type(image))
#     print('Label type: ' , type(label))
#     print('inside read_image func: ',image)
    return image,label

In [10]:
def augment(image,label):
    #print('augment func image type: ',type(image))
#     print('augment func label type',type(label))
#     print(type(image))
#     print(image)
#     print(label)
    #plt.imshow(image)
    image=tf.image.resize(
    image,
    (224,224),
    preserve_aspect_ratio=False,
    antialias=False,
    name=None)
    
    return image,label


In [11]:
# Do not delete
val_data=ds_train.map(read_image).map(augment).batch(16)

# extracting accuracy of Validation dataset

In [ ]:
#11.42am start the line below

In [12]:
# import tensorflow as tf
# tf.config.list_physical_devices('GPU')
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# #print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [13]:
model=tf.keras.applications.vgg16.VGG16(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)


In [15]:

count=0
with tf.device('/gpu:0'):
    for x_val,y_val in tqdm(val_data):
        example_1_image=x_val
        label_1_image=y_val
        prediction_16_batch=model(example_1_image*255)
        top_1_prediction=tf.math.argmax(prediction_16_batch,axis=1)
        count+=(top_1_prediction==label_1_image).numpy().sum()


  0%|                                                                                          | 0/3125 [00:00<?, ?it/s]2022-08-15 23:26:36.477607: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-08-15 23:26:36.497997: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3699850000 Hz
2022-08-15 23:26:36.633144: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-08-15 23:26:37.409148: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-08-15 23:26:37.477216: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-08-15 23:26:38.258691: I tensorflow/stream_executor/platform/defau

In [16]:
top_1_accuracy=(count/50000)*100
print(f"Top 1 accuracy is: {top_1_accuracy}%")

Top 1 accuracy is: 42.931999999999995%


In [39]:
(model.predict(example_1_image*255)==model(example_1_image*255)).numpy().sum()

16000

In [60]:
#top 5 prediction
# predict_alps=model(image_alps_input*255)
# predict_alps.shape
# prediction_by_index_descending_order=tf.argsort(
#     predict_alps, axis=-1, direction='DESCENDING', stable=False, name=None
# )
# type(prediction_by_index_descending_order)
# tf.slice(prediction_by_index_descending_order[0], begin=[0], size=[5])
iter_num=0
count=0
total_images=0
with tf.device('/gpu:0'):
    for x_val,y_val in tqdm(val_data):
#         if iter_num==15:
#             break
#         else:
        example_1_image=x_val
        label_1_image=y_val
        prediction_16_batch=model.predict(example_1_image*255) #shape is 16,1000
        total_images=prediction_16_batch.shape[0]
        prediction_by_index_descending_order=tf.argsort(prediction_16_batch, axis=-1, direction='DESCENDING', stable=False, name=None)
        #top_5
        top_5_prediction=prediction_by_index_descending_order[:,:5]
        label_1_image=tf.cast(label_1_image, tf.int32)
        prediction_one_hot_1=tf.one_hot(top_5_prediction,1000,on_value=None,off_value=None,axis=1,dtype=None,name=None)
        prediction_one_hot_2=tf.math.reduce_sum(prediction_one_hot_1, axis=2, keepdims=False, name=None)

        ground_truth=tf.one_hot(label_1_image,1000)

        final=ground_truth*prediction_one_hot_2
        #print(final)
        add_to_count=tf.math.reduce_sum(final, keepdims=False, name=None)
        print(add_to_count,total_images)
        print(f"current batch accuracy is: {(add_to_count/total_images)*100}%")
        count+=add_to_count
        iter_num+=1
        


  0%|                                                                                  | 3/3125 [00:00<05:23,  9.66it/s]

tf.Tensor(14.0, shape=(), dtype=float32) 16
current batch accuracy is: 87.5%
tf.Tensor(13.0, shape=(), dtype=float32) 16
current batch accuracy is: 81.25%
tf.Tensor(10.0, shape=(), dtype=float32) 16
current batch accuracy is: 62.5%


  0%|▏                                                                                 | 5/3125 [00:00<05:15,  9.89it/s]

tf.Tensor(10.0, shape=(), dtype=float32) 16
current batch accuracy is: 62.5%
tf.Tensor(9.0, shape=(), dtype=float32) 16
current batch accuracy is: 56.25%
tf.Tensor(11.0, shape=(), dtype=float32) 16
current batch accuracy is: 68.75%


  0%|▏                                                                                 | 9/3125 [00:00<04:12, 12.33it/s]

...

tf.Tensor(14.0, shape=(), dtype=float32) 16
current batch accuracy is: 87.5%
tf.Tensor(10.0, shape=(), dtype=float32) 16
current batch accuracy is: 62.5%
tf.Tensor(12.0, shape=(), dtype=float32) 16
current batch accuracy is: 75.0%


100%|██████████████████████████████████████████████████████████████████████████████▉| 3123/3125 [03:46<00:00, 12.81it/s]

tf.Tensor(9.0, shape=(), dtype=float32) 16
current batch accuracy is: 56.25%
tf.Tensor(9.0, shape=(), dtype=float32) 16
current batch accuracy is: 56.25%


100%|███████████████████████████████████████████████████████████████████████████████| 3125/3125 [03:46<00:00, 13.79it/s]

tf.Tensor(14.0, shape=(), dtype=float32) 16
current batch accuracy is: 87.5%


In [62]:
count/50000

<tf.Tensor: shape=(), dtype=float32, numpy=0.67884>

In [58]:
label_1_image

<tf.Tensor: shape=(16,), dtype=int32, numpy=
array([ 25, 463, 215, 173,  35, 741, 125, 787, 289, 425, 973,   1, 167,
       121, 445, 702], dtype=int32)>

In [59]:
top_5_prediction

<tf.Tensor: shape=(16, 5), dtype=int32, numpy=
array([[ 71, 310, 319, 971, 983],
       [543, 422, 427, 720, 756],
       [162, 166, 215, 212, 176],
       [224, 537, 177, 248, 250],
       [947, 452, 929, 328, 793],
       [741, 750, 611, 406, 539],
       [728, 879, 125, 982, 601],
       [ 80, 898, 455, 818, 862],
       [289, 288, 290, 287, 285],
       [425, 497, 716, 580, 912],
       [973, 938, 107, 396, 109],
       [107, 149, 132,   5, 126],
       [167, 166, 246, 210, 212],
       [845, 971, 731, 898, 903],
       [562,  71,  58, 327, 712],
       [422, 602, 702, 543, 416]], dtype=int32)>

In [18]:
top_5_accuracy=(count/total_images)*100
print(f"The top 5 accuracy score is: {top_5_accuracy}%")

The top 5 accuracy score is: 67.88399505615234%


In [19]:
total_images

50000

In [20]:
label_1_image

<tf.Tensor: shape=(16,), dtype=int32, numpy=
array([367, 430,  44,  81, 399,  24, 120, 357, 826, 101, 644, 283,  26,
       232, 982, 355], dtype=int32)>

In [21]:
top_5_prediction

<tf.Tensor: shape=(16, 5), dtype=int32, numpy=
array([[365, 366, 373, 367, 371],
       [430, 890, 842, 611, 416],
       [ 44,  41,  46,  40,  38],
       [330, 331,  81,  86,  85],
       [797, 470, 692, 885, 399],
       [ 24,  85,  82, 330,  83],
       [120, 124,  71,  77, 119],
       [357, 356, 336, 298, 358],
       [826, 530, 531, 426, 778],
       [385, 101, 386, 178, 209],
       [677, 644, 702, 733, 704],
       [283, 287, 332, 203, 281],
       [ 48,  39,  43,  42,  46],
       [224, 223, 205, 256, 206],
       [452, 431, 982, 516, 999],
       [355, 181, 229, 354, 348]], dtype=int32)>

# how to get top 5 predictions

In [ ]:
predict_alps=model(image_alps_input*255)
predict_alps.shape
prediction_by_index_descending_order=tf.argsort(
    predict_alps, axis=-1, direction='DESCENDING', stable=False, name=None
)
type(prediction_by_index_descending_order)
tf.slice(prediction_by_index_descending_order[0], begin=[0], size=[5])
# tf.math.argmax(
#     predict_alps,
#     axis=1,
#     output_type=tf.dtypes.int64,
#     name=None
# )

# End of how to get top 5 predictions

In [ ]:
predict_on_2_images=model(example_1_image*255)
predict_on_2_images.shape

In [ ]:
tf.math.argmax(
    predict_on_2_images,
    axis=1,
    output_type=tf.dtypes.int64,
    name=None
)

In [ ]:
label_1_image

In [ ]:
plt.imshow(example_1_image[7])

# Castle Image

In [ ]:
img = cv2.imread('/home/charlene/tensorflow_1/examples/images/castle.jpg')
input_img=np.expand_dims(img,0)
input_img.shape

In [ ]:
# mean=[0.485, 0.456, 0.406]
# std=[0.229, 0.224, 0.225]

In [ ]:
# try_input_img=((img/255.0)-mean)/std
# try_input_img_1=np.expand_dims(try_input_img,0)
# try_input_img_1.shape

In [ ]:
predict2=model(input_img)
tf.math.argmax(
    predict2,
    axis=1,
    output_type=tf.dtypes.int64,
    name=None
)